In [1]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import pandas as pd
import numpy as np
from copy import deepcopy

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")

def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"
    
# #obtain the minimum transit travel time between all modes.
# #uses outputs from HDR transit time toolbox
# mcSpec = {
#     "expression": None,
#     "result": "",
#     "constraint": {
#         "by_value": None,
#         "by_zone": None
#     },
#     "aggregation": {
#         "origins": None,
#         "destinations": None
#     },
#     "type": "MATRIX_CALCULATION"
# }
# def minimum_transit_time(tt_dir, min_transit_time,scenario,go_sub_mtx_nums):
    
#     #import the .311 files containing GO Train and subway OD travel times
#     for root, dirs, files in os.walk(tt_dir):
#         for matrix_file in files:
#             if matrix_file.endswith(".311"):
#                 matrix_transaction(os.path.join(root, matrix_file),scenario)
                
#     create_matrix(matrix_id = min_transit_time,
#         matrix_name= "min.transit.time",
#         matrix_description = "minimum.transit.time")

#     matrix_list ={ "express.bus.actual": "",
#                    "rapid.bus.actual":"",
#                    "local.bus.actual":""
#                    }

#     #Import matrices
#     for matrix_name in matrix_list.keys():
#         matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
#         create_matrix(matrix_id = matrix_list[matrix_name],
#                       matrix_name= matrix_name,
#                       matrix_description = matrix_name)

#     import_matrices_from_directory(tt_dir, matrix_list, ".mdf", scenario)
    
#     spec = {
#            "type": "MATRIX_CALCULATION",
#            "result": min_transit_time,
#            "expression":  "(" + matrix_list["express.bus.actual"] + ".min."\
#             + matrix_list["rapid.bus.actual"] + ".min."\
#             + matrix_list["local.bus.actual"] + ")"
#     }
#     matrix_calculator(spec)

#     for mtx_num in go_sub_mtx_nums:
#         expression = min_transit_time + ".min.mf" + mtx_num + "*(mf" + mtx_num + "!=0)+" + min_transit_time + "*(mf" + mtx_num + "==0)"
#         spec = {
#            "type": "MATRIX_CALCULATION",
#            "result": min_transit_time,
#            "expression":  expression
#         }
#         matrix_calculator(spec)

#     print "Finished Calculating OD Transit Travel Times"
    
# def minimum_transit_time_mto(tt_dir,mto_skims, min_transit_time, scenario):            
#     create_matrix(matrix_id = min_transit_time,
#         matrix_name= "min.transit.time",
#         matrix_description = "minimum.transit.time")

#     matrix_list ={"express.bus.actual": "",
#                    "rapid.bus.actual":"",
#                    "local.bus.actual":""
#                    }
    
#     mat_list_mto = {"Express Bus": "",
#                    "Local Bus": "",
#                    "Rapid Bus": "",
#                    "Subway": "",
#                    "Train": ""}

#     matching = {"Express Bus": "express.bus.actual",
#                "Local Bus":"local.bus.actual",
#                "Rapid Bus":"rapid.bus.actual"}
#     #Import matrices
#     for matrix_name in matrix_list.keys():
#         matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
#         create_matrix(matrix_id = matrix_list[matrix_name],
#                       matrix_name= matrix_name,
#                       matrix_description = matrix_name)

#     import_matrices_from_directory(tt_dir, matrix_list, ".mdf", scenario)
    
#     for matrix_name in mat_list_mto.keys():
#         mat_list_mto[matrix_name] = eb.available_matrix_identifier('FULL')
#         create_matrix(matrix_id = mat_list_mto[matrix_name],
#                   matrix_name= matrix_name.replace(" ","_"),
#                   matrix_description = matrix_name)

#     import_matrices_from_directory(mto_skims, mat_list_mto, ".mdf", scenario)
    
#     for match in matching.keys():
#         mto = mat_list_mto[match]
#         hdr = matrix_list[matching[match]]
#         spec = {
#            "type": "MATRIX_CALCULATION",
#            "result": mto,
#            "expression":  hdr,
#              "constraint": {
#         "by_value": {
#             "interval_min": 0,
#             "interval_max": 0,
#             "condition": "INCLUDE",
#             "od_values": mto
#         },
#         "by_zone": None}
#             }
#         matrix_calculator(spec)
#     min_exp = "("
#     for matrix_name in mat_list_mto.keys():
#         mat_id = mat_list_mto[matrix_name]
#         min_exp = min_exp + mat_id + ".min."
#         spec = {
#            "type": "MATRIX_CALCULATION",
#            "result": mat_id,
#            "expression":  "9999",
#              "constraint": {
#         "by_value": {
#             "interval_min": 0,
#             "interval_max": 0,
#             "condition": "INCLUDE",
#             "od_values": mat_id
#         },
#         "by_zone": None}
#             }
#         matrix_calculator(spec)
#     min_exp = min_exp[0:-5] + ")"

#     spec = {
#            "type": "MATRIX_CALCULATION",
#            "result": min_transit_time,
#            "expression":min_exp}
#     matrix_calculator(spec)        
#     spec = {
#            "type": "MATRIX_CALCULATION",
#            "result": min_transit_time,
#            "expression":  "0",
#         "constraint": {
#         "by_value": {
#             "interval_min": 9999,
#             "interval_max": 9999,
#             "condition": "INCLUDE",
#             "od_values": min_transit_time
#         },
#         "by_zone": None}
#             }
#     matrix_calculator(spec)
#     print "Finished Calculating OD Transit Travel Times"

def get_mc_spec(result, expression, agg_o, agg_d, con_o, con_d, con_val):
    specification = {
        "type": "MATRIX_CALCULATION",
        "result": result,
        "expression":  expression,
        "aggregation": {
            "origins": agg_o,
            "destinations": agg_d
        },
        "constraint": {
            "by_value": con_val,
            "by_zone": {
                "origins": con_o,
                "destinations": con_d
            }
        }
    }
    return specification

def MOE_fraction(out_file,numerator,denominator,zones_array):
    
    for origins in zones_array:
        spec = get_mc_spec(None, denominator,"+","+",origins,"all",None)
        bottom = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, numerator,"+","+",origins,"all",None)
        top = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(top/bottom))
    out_file.write("\n")
netCalcSpec = {
    "result": None,
    "expression": None,
    "aggregation": None,
    "selections": {
        "link": None
    },
    "type": "NETWORK_CALCULATION"
}

modes = {"sov":['trips_peak_hbw_auto1_nt', 'trips_peak_hbw_auto1_toll'],
        "hov2": ['trips_peak_hbw_auto2_nt_hov', 'trips_peak_hbw_auto2_nt_nh',
       'trips_peak_hbw_auto2_toll_hov', 'trips_peak_hbw_auto2_toll_nh'],
        "hov3": ['trips_peak_hbw_auto3_nt_hov', 'trips_peak_hbw_auto3_nt_nh',
       'trips_peak_hbw_auto3_toll_hov', 'trips_peak_hbw_auto3_toll_nh'],
        "Express Bus": ['trips_peak_hbw_transit_express_bus_walk','od_trips_peak_hbw_transit_express_bus_drive'],
        "Rapid Bus": ['trips_peak_hbw_transit_rapid_bus_walk','trips_peak_hbw_transit_rapid_bus_drive'],
        "Local Bus": ['trips_peak_hbw_transit_local_bus_walk','trips_peak_hbw_transit_local_bus_drive'],
        "Train":['od_trips_peak_hbw_transit_train_walk_access',
       'od_trips_peak_hbw_transit_train_transit_access',
       'od_trips_peak_hbw_transit_train_park_access',
       'od_trips_peak_hbw_transit_train_drop_off'],
        "Subway": ['od_trips_peak_hbw_transit_subway_walk_access',
       'od_trips_peak_hbw_transit_subway_transit_access',
       'od_trips_peak_hbw_transit_subway_park_access',
       'od_trips_peak_hbw_transit_subway_drop_off'],
        "walk": ['trips_peak_hbw_active_walk'],
        "bike": ['trips_peak_hbw_active_bike']}

In [2]:
#scenario folder names

#BAU
sc_2051_BAU_ogs = "2051\\Short List\\BAU\\BAU_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201127"
sc_2051_prefnet_ogs = "2051\\Preferred Network\\Preferred_Network_SList_APTG_2051_GGH_MTP_V106A3_CS222_20210305"
sc_2041_BAU_ogs = "2041\\GGHM Base Case\\USE - Sent Through FTP September 20 2018"
sc_2016_BAU_ogs = "2016"

In [3]:
## zone grouping definitions ## 
zones_pd1 = "gf1"
zones_airports = "ga1"
zones_ugc = "gb4"
zones_inner = "gc1"
zones_outer = "gc2"
zones_university = "gd1"
zones_employment = "ge1"
zones_hospitals = "gh1"
shed_central = "gs10;gs11"
rural_central = "gs11"
shed_east = "gs20;gs21"
rural_east = "gs21"
shed_midwest = "gs30;gs31"
rural_midwest = "gs31"
shed_southwest = "gs40;gs41"
rural_southwest = "gs41"
shed_west = "gs50;gs51"
rural_west = "gs51"
shed_north = "gs60;gs61"
rural_north = "gs61"
shed_niagara = "gs70;gs71"
rural_niagara = "gs71"
shed_urban = "gs10;gs20;gs30;gs40;gs50;gs60;gs70"
shed_rural = "gs11;gs21;gs31;gs41;gs51;gs61;gs71"
urban_lu = "gb2;gb3;gb4"

#auto occupancy
sov_oc = 1.2
hov2_oc = 2.2
hov3_oc = 3.3

#number of hours for transit assignment
num_hrs_am = 2

In [5]:
pop_o_matrix_ogs = "mo19"
emp_o_matrix_ogs = "mo20"
senior_o_matrix_ogs = "mo21"
low_inc_o_matrix_ogs = "mo22"# number of low inc households
youth_o_matrix_ogs = "mo23"
pop_d_matrix_ogs = "md92"
emp_d_matrix_ogs = "md93"
low_income_zones_ogs = "mo24"

pop_o_matrix_luv1 = "mo25"
emp_o_matrix_luv1 = "mo26"
senior_o_matrix_luv1 = "mo27"
low_inc_o_matrix_luv1 = "mo28"# number of low inc households
youth_o_matrix_luv1 = "mo29"
pop_d_matrix_luv1 = "md94"
emp_d_matrix_luv1 = "md95"
low_income_zones_luv1 = "mo30"

pop_o_matrix_luv2 = "mo31"
emp_o_matrix_luv2 = "mo32"
senior_o_matrix_luv2 = "mo33"
low_inc_o_matrix_luv2 = "mo34"# number of low inc households
youth_o_matrix_luv2 = "mo35"
pop_d_matrix_luv2 = "md96"
emp_d_matrix_luv2 = "md97"
low_income_zones_luv2 = "mo36"

pop_o_matrix_luv3 = "mo37"
emp_o_matrix_luv3 = "mo38"
senior_o_matrix_luv3 = "mo39"
low_inc_o_matrix_luv3 = "mo40"# number of low inc households
youth_o_matrix_luv3 = "mo41"
pop_d_matrix_luv3 = "md98"
emp_d_matrix_luv3 = "md99"
low_income_zones_luv3 = "mo42"

hospital_d = "md90"
postsec_d = "md91"
parking_d = "md24"

truck_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Long List\BAU\MTO_2051_GGH_MTP_5002_V106A3_HS032_20200117\Assignable matrices"
tt_dir = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Peak"
tt_dir_op = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\HDR_transit_time_results\20200212_2051_BAU_HS - Off-Peak"
hdr_bau_peak = "mf160"
hdr_bau_offpeak = "mf170"
local_fare_mat = "mf180"

In [11]:
scens = {"2016_prefnet_OGS":[sc_2016_BAU_ogs,[100,101],[pop_o_matrix_ogs,emp_o_matrix_ogs,senior_o_matrix_ogs,
                                                low_inc_o_matrix_ogs,youth_o_matrix_ogs,pop_d_matrix_ogs,
                                                emp_d_matrix_ogs,low_income_zones_ogs]]} #AM,MD

In [29]:
#Matrix naming for inputs

#peak period - 7 hrs
sov_demand_pk = "trips_peak_sov"
hov2_demand_pk = "trips_peak_hov2"
hov3_demand_pk = "trips_peak_hov3pl"

sov_demand_offpk = "trips_offpeak_sov"
hov2_demand_offpk = "trips_offpeak_hov2"
hov3_demand_offpk = "trips_offpeak_hov3pl"

active_demand_daily = "trips_active_daily"
transit_demand_daily = "trips_transit_daily"

# distance
sov_dist_am = "skim.am_peak.auto.sov.dist"
sov_dist_md = "skim.midday.auto.sov.dist"

# travel time
sov_time_am = "skim.am_peak.auto.sov.time"
hov2_time_am = "skim.am_peak.auto.hov2.time"
hov3_time_am = "skim.am_peak.auto.hov3.time"


In [30]:
zones_array = ["all",zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]
rural_array = [shed_rural,zones_pd1,rural_central,rural_east,rural_midwest,rural_southwest,rural_west,rural_north,rural_niagara]
zones_array_2 = [zones_pd1,shed_central,shed_east,shed_midwest,shed_southwest,shed_west,shed_north,shed_niagara]

In [31]:
def MOE_fraction_min(out_file,num1,den1,num2,den2,zones_array):
    
    for origins in zones_array:
        spec = get_mc_spec(None, den1,"+","+",origins,"all",None)
        bottom1 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num1,"+","+",origins,"all",None)
        top1 = float(matrix_calculator(spec)['result'])
        
        spec = get_mc_spec(None, den2,"+","+",origins,"all",None)
        bottom2 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num2,"+","+",origins,"all",None)
        top2 = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(min(top1/bottom1,top2/bottom2)))
    out_file.write("\n")
        
def MOE_fraction_max(out_file,num1,den1,num2,den2,zones_array):
    
    for origins in zones_array:
        spec = get_mc_spec(None, den1,"+","+",origins,"all",None)
        bottom1 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num1,"+","+",origins,"all",None)
        top1 = float(matrix_calculator(spec)['result'])
        
        spec = get_mc_spec(None, den2,"+","+",origins,"all",None)
        bottom2 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num2,"+","+",origins,"all",None)
        top2 = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(max(top1/bottom1,top2/bottom2)))
    out_file.write("\n")
    
def MOE_fraction_avg(out_file,num1,den1,num2,den2,zones_array):
    
    for origins in zones_array:
        spec = get_mc_spec(None, den1,"+","+",origins,"all",None)
        bottom1 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num1,"+","+",origins,"all",None)
        top1 = float(matrix_calculator(spec)['result'])
        
        spec = get_mc_spec(None, den2,"+","+",origins,"all",None)
        bottom2 = matrix_calculator(spec)['result']
        spec = get_mc_spec(None, num2,"+","+",origins,"all",None)
        top2 = float(matrix_calculator(spec)['result'])
        out_file.write("{0:.4f},".format(((top1/bottom1)+(top2/bottom2))/2))
    out_file.write("\n")
        
truck_generators_gateways = "4182;6019;gt2;gt3;gt4;gt5" # only pearson and hamilton for airports

borders = "gt4"

truck_generators = "4182;6019;gt2;gt3"# only pearson and hamilton for airports

airports = "4182;6019"


roadCosts = {1: [5000000,4500000], # 2 lane widening
            2: [10000000,9000000], # 4 lane widening
            3: [15000000,13500000], # 6 lane widening
            4: [8375000,5750000], # 2 lane new construction
            5: [11875000,9250000], # 4 lane new construction
            6: [15375000,12750000], # 6 lane new construction
            7: [500000000,500000000], # 401 tunnel/overpass cost - Assumed it would be triple the cost of a normal 8 lane new construction
            8: [2500000,2500000], # Core/collector conversion cost
            9: [18875000,16250000]} # 8 lane new construction

transitCosts = {1: 12500000, # GO Train New Corridor (freight track only)
            2: 12500000, # GO Train New Corridor (no existing track)
            3: 5000000, # Go Train all day service and 15 minute
            4: 250000000, # Subway
            5: 62500000, # LRT
            6: 12000000, # BRT Lite
            8: 3500000, # GO train all day service
            9: 1500000, # GO train 15 minute service
            10: 5750000, #Ferry
            13: 25000000}  #BRT Dedicated Facility

bridgeCost = {1: [29000000,29000000],# Assumes 10000 m2 bridge, using prestressed concrete, as these will be wide highway bridges
             2: [205000000,190000000], # 4 leg interchange
             3: [80000000,75000000], # 3 leg interchange
             4: [100000000,30000000], # GO Station
             5: [5000000000,5000000000], #Missing Link cost
             6: [2500000000,2500000000], # Line 1/2 Capacity expansion surplus cost
             7: [100000000,100000000],#Kitchener Pearson realignment surplus cost
             8: [500000000,50000000]}  #Expensive Skyway Bridge

#Based on parametric estimating guide. This is a surplus cost - not per km of bridge

In [38]:
for scen in scens.keys():
    print("Now processing {}".format(scen))
    scenario_folder = scens[scen][0]
    matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs" + "\\" + scenario_folder
    mto_tt_skims = matrix_folder + "\\Peak minimum skims LVL3_PATH\\time_all"
    mto_tt_skims_op = matrix_folder + "\\Offpeak minimum skims LVL3_PATH\\time_all"
    pop_o_matrix = scens[scen][2][0]
    emp_o_matrix = scens[scen][2][1]
    senior_o_matrix = scens[scen][2][2]
    low_inc_o_matrix = scens[scen][2][3]
    youth_o_matrix = scens[scen][2][4]
    pop_d_matrix = scens[scen][2][5]
    emp_d_matrix = scens[scen][2][6]
    low_income_zones = scens[scen][2][7]
    
    sc_num_am = scens[scen][1][0]
    sc_num_md = scens[scen][1][1]
#     sc_num_pm = scens[scen][1][2]
    #scenarios
    sc_am = eb.scenario(sc_num_am)
    sc_md = eb.scenario(sc_num_md)
#     sc_pm = eb.scenario(sc_num_pm)
    #networks
    ntwk_am = sc_am.get_network()
    ntwk_md = sc_md.get_network()
#     ntwk_pm = sc_pm.get_network()
    
#     clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")
    
#     min_transit_time_peak = "mf140"
    
#     #minimum transit time stuff
#     create_matrix(matrix_id = min_transit_time_peak,
#         matrix_name= "min.transit.time",
#         matrix_description = "minimum.transit.time")

#     mat_list_mto = {"All Transit": ""}

#     for matrix_name in mat_list_mto.keys():
#         mat_list_mto[matrix_name] = eb.available_matrix_identifier('FULL')
#         create_matrix(matrix_id = mat_list_mto[matrix_name],
#                   matrix_name= matrix_name.replace(" ","_"),
#                   matrix_description = matrix_name)

#     import_matrices_from_directory(mto_tt_skims, mat_list_mto, ".mdf", sc_am)

#     spec = {
#        "type": "MATRIX_CALCULATION",
#        "result": min_transit_time_peak,
#        "expression":  mat_list_mto["All Transit"],
#          "constraint": {
#     "by_value": None,
#     "by_zone": None}
#         }
#     matrix_calculator(spec)

#     spec2 = {
#        "type": "MATRIX_CALCULATION",
#        "result": min_transit_time_peak,
#        "expression":  hdr_bau_peak,
#          "constraint": {
#     "by_value": {
#         "interval_min": 0,
#         "interval_max": 0,
#         "condition": "INCLUDE",
#         "od_values":  min_transit_time_peak
#     },
#     "by_zone": None}
#         }
#     matrix_calculator(spec2)
    
#     spec3 = {
#        "type": "MATRIX_CALCULATION",
#        "result": min_transit_time_peak,
#        "expression":  hdr_bau_peak,
#          "constraint": {
#     "by_value": {
#         "interval_min": 0,
#         "interval_max": 800,
#         "condition": "EXCLUDE",
#         "od_values":  min_transit_time_peak
#     },
#     "by_zone": None}
#         }
#     matrix_calculator(spec3)

#     clear_matrices(eb,[min_transit_time_peak,hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")

#     min_transit_time_offpeak = "mf150"

#     create_matrix(matrix_id = min_transit_time_offpeak,
#         matrix_name= "min.transit.time.op",
#         matrix_description = "minimum.transit.time")

#     mat_list_mto = {"All Transit": ""}

#     for matrix_name in mat_list_mto.keys():
#         mat_list_mto[matrix_name] = eb.available_matrix_identifier('FULL')
#         create_matrix(matrix_id = mat_list_mto[matrix_name],
#                   matrix_name= matrix_name.replace(" ","_"),
#                   matrix_description = matrix_name)

#     import_matrices_from_directory(mto_tt_skims_op, mat_list_mto, ".mdf", sc_md)

#     spec = {
#        "type": "MATRIX_CALCULATION",
#        "result": min_transit_time_offpeak,
#        "expression":  mat_list_mto["All Transit"],
#          "constraint": {
#     "by_value": None,
#     "by_zone": None}
#         }
#     matrix_calculator(spec)

#     spec2 = {
#        "type": "MATRIX_CALCULATION",
#        "result": min_transit_time_offpeak,
#        "expression":  hdr_bau_offpeak,
#          "constraint": {
#     "by_value": {
#         "interval_min": 0,
#         "interval_max": 0,
#         "condition": "INCLUDE",
#         "od_values":  min_transit_time_offpeak
#     },
#     "by_zone": None}
#         }
#     matrix_calculator(spec2)
    
#     spec3 = {
#        "type": "MATRIX_CALCULATION",
#        "result": min_transit_time_offpeak,
#        "expression":  hdr_bau_offpeak,
#          "constraint": {
#     "by_value": {
#         "interval_min": 0,
#         "interval_max": 800,
#         "condition": "EXCLUDE",
#         "od_values":  min_transit_time_offpeak
#     },
#     "by_zone": None}
#         }
#     matrix_calculator(spec3)

#     clear_matrices(eb,[min_transit_time_peak,min_transit_time_offpeak, hdr_bau_peak, hdr_bau_offpeak,local_fare_mat],"FULL")
# ### matrices
#     matrix_list = {
#         sov_demand_pk : "",
#         hov2_demand_pk : "",
#         hov3_demand_pk : "",
#         sov_demand_offpk : "",
#         hov2_demand_offpk : "",
#         hov3_demand_offpk : "",
#         sov_dist_am : "",
#         sov_time_am: "",
#         hov2_time_am: "",
#         hov3_time_am : "",
#         sov_dist_md:"",
#         active_demand_daily:"",
#         transit_demand_daily:"",
#     }

#     #Import matrices
#     for matrix_name in matrix_list.keys():
#         matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
#         print matrix_name
#         create_matrix(matrix_id = matrix_list[matrix_name],
#                       matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
#                       matrix_description = matrix_name)

#     import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_am) 

#     truck_list = {truck_h_demand_am: '',
#      truck_l_demand_am: '',
#      truck_m_demand_am: '',
#      truck_h_demand_md: '',
#      truck_l_demand_md: '',
#      truck_m_demand_md: ''}

#     for matrix_name in truck_list.keys():
#         truck_list[matrix_name] = eb.available_matrix_identifier('FULL')
#         print matrix_name
#         create_matrix(matrix_id = truck_list[matrix_name],
#                       matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
#                       matrix_description = matrix_name)

#     import_matrices_from_directory(truck_folder, truck_list, ".mdf",401) 

#     matrix_list.update(truck_list)
    
    #Expressions
    #total person trips - Peak Period, Offpeak period
    
    exp_pk_pers_trips_auto = "(" + matrix_list[hov2_demand_pk] \
        + "+" + matrix_list[hov3_demand_pk] \
        + "+" + matrix_list[sov_demand_pk] + ")"

    exp_offpk_pers_trips_auto = "(" + matrix_list[hov2_demand_offpk] \
        + "+" + matrix_list[hov3_demand_offpk] \
        + "+" + matrix_list[sov_demand_offpk]  + ")"
    
    exp_daily_pers_transtrips = "(" + matrix_list[transit_demand_daily] + ")"
    exp_daily_pers_activetrips = "(" + matrix_list[active_demand_daily] + ")"

#     exp_pk_pers_trips_auto = "(" + matrix_list[hov2_demand_pk] \
#         + "+" + matrix_list[hov3_demand_pk] \
#         + "+" + matrix_list[sov_demand_pk] + ")"

#     exp_offpk_pers_trips_auto = "(" + matrix_list[hov2_demand_offpk] \
#         + "+" + matrix_list[hov3_demand_offpk] \
#         + "+" + matrix_list[sov_demand_offpk]  + ")"

    exp_pk_pers_autotrip_speed = "(60*" + matrix_list[sov_dist_am]\
        + "*(" + matrix_list[hov2_demand_pk] + "/" + matrix_list[hov2_time_am]+ "*(" + matrix_list[hov2_time_am] + "!=0)+"\
        + matrix_list[hov3_demand_pk]  + "/" + matrix_list[hov3_time_am]+ "*(" + matrix_list[hov3_time_am] + "!=0)+"\
        + matrix_list[sov_demand_pk] + "/" + matrix_list[sov_time_am]+ "*(" + matrix_list[sov_time_am] + "!=0)))"
    
#     exp_pk_pers_autotrip_speed = "(60*" + matrix_list[sov_dist_am]+ "/" + matrix_list[sov_time_am]\
#         + "*(" + matrix_list[sov_demand_pk] + "+" + matrix_list[hov2_demand_pk]\
#     + "+" + matrix_list[hov3_demand_pk] + ")*(" + matrix_list[sov_time_am] + "!=0))"
    
#     exp_offpk_pers_trip_speed = "(60*" + matrix_list[sov_dist_am]\
#         + "*(" + matrix_list[hov2_demand_offpk] + "/" + matrix_list[hov2_time_md]+ "*(" + matrix_list[hov2_time_md] + "!=0)+"\
#         + matrix_list[hov3_demand_offpk]  + "/" + matrix_list[hov3_time_md]+ "*(" + matrix_list[hov3_time_md] + "!=0)+"\
#         + matrix_list[sov_demand_offpk] + "/" + matrix_list[sov_time_md]+ "*(" + matrix_list[sov_time_md] + "!=0))"
    
    #Shed VKT Section

    ntwks = [ntwk_am,ntwk_md]

    zones_vkt = [{"all":{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
                 zones_pd1:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
                 shed_central:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
                 shed_east:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
                 shed_midwest:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
                 shed_southwest:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
                 shed_west:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
                 shed_north:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0},
                 shed_niagara:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0, "cost":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0,"bridges":0.0}},
                {"all":{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
                 zones_pd1:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
                 shed_central:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
                 shed_east:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
                 shed_midwest:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
                 shed_southwest:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
                 shed_west:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
                 shed_north:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0},
                 shed_niagara:{"vkt": 0.0, "cong_vkt": 0.0, "truck_vkt": 0.0,"cong_truck_vkt":0.0,
                        "pht": 0.0, "ff_pht":0.0, "trk_pht":0.0,"ff_trk_pht":0.0}}]

    cong = 0.85
    truck_vols = ["@volau_htruck","@volau_mtruck","@volau_ltruck"]
    auto_vols = ["@volau1_nt","@volau1_toll",\
                 "@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov",\
                "@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]
    
    sov_vols = ["@volau1_nt","@volau1_toll"]
    hov2_vols = ["@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov"]
    hov3_vols = ["@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]

    for i in range(2):
        for link in ntwks[i].links():
            add = False 
            #This section determines what shed the link belongs to
            if ((link.type > 860 and link.type < 900) or (link.type >= 325 and link.type <= 328) or link.type == 330 or link.type == 332): #North
                shed = shed_north
                add = True
            elif ((link.type > 101 and link.type <200) or link.type == 100 or link.type ==333 or link.type ==329 or link.type ==331): #Central
                shed = shed_central
                add = True
            elif link.type == 101: #PD1
                shed = zones_pd1
                add = True
            elif ((link.type >199 and link.type <300) or (link.type > 899 and link.type < 950)): # East
                shed = shed_east
                add = True
            elif link.type == 820 or link.type ==850: # West
                shed = shed_west
                add = True
            elif (link.type >599 and link.type<700) or link.type ==810 or link.type == 800: #Southwest
                shed = shed_southwest
                add = True
            elif link.type == 700: # Niagara
                shed = shed_niagara
                add = True
            elif link.type >= 400 and link.type < 600: # Midwest
                shed = shed_midwest
                add = True

            if add:
#                 if i == 0: #Only need to do cost for AM network
#                     if link["@rcost"] > 0:
#                         r_imp = link["@rcost"]
#                         if shed == shed_central or shed == zones_pd1 or shed == shed_midwest or shed == shed_southwest:
#                             zones_vkt[i]["all"]["cost"] += roadCosts[r_imp][0]*link.length
#                             zones_vkt[i][shed]["cost"] += roadCosts[r_imp][0]*link.length
#                         else:
#                             zones_vkt[i]["all"]["cost"] += roadCosts[r_imp][1]*link.length
#                             zones_vkt[i][shed]["cost"] += roadCosts[r_imp][1]*link.length
#                     if link["@tcost"] > 0:
#                         t_imp = link["@tcost"]
#                         zones_vkt[i]["all"]["cost"] += transitCosts[t_imp]*link.length
#                         zones_vkt[i][shed]["cost"] += transitCosts[t_imp]*link.length
                            
#                     if link["@bcost"] > 0:
#                         b_imp = link["@bcost"]
#                         if b_imp == 1 or b_imp == 8:
#                             zones_vkt[i]["all"]["bridges"] += 1
#                             zones_vkt[i][shed]["bridges"] += 1
#                         if b_imp == 4: # GO Station case
#                             if (link.type > 101 and link.type <200):
#                                 zones_vkt[i]["all"]["cost"] += bridgeCost[b_imp][0]
#                                 zones_vkt[i][shed]["cost"] += bridgeCost[b_imp][0]
#                             else:
#                                 zones_vkt[i]["all"]["cost"] += bridgeCost[b_imp][1]
#                                 zones_vkt[i][shed]["cost"] += bridgeCost[b_imp][1]
#                         else:
#                             if shed == shed_central or shed == zones_pd1 or shed == shed_midwest or shed == shed_southwest:
#                                 zones_vkt[i]["all"]["cost"] += bridgeCost[b_imp][0]
#                                 zones_vkt[i][shed]["cost"] += bridgeCost[b_imp][0]
#                             else:
#                                 zones_vkt[i]["all"]["cost"] += bridgeCost[b_imp][1]
#                                 zones_vkt[i][shed]["cost"] += bridgeCost[b_imp][1]

                if link.volume_delay_func > 0 and link.volume_delay_func < 90 and ntwks[i].mode("I") in link.modes:
                    volume = 0.0
                    person_vol =0.0
                    truck_volume = 0.0
                    
                    for vol in sov_vols:
                        volume += link[vol]
                        person_vol += link[vol]*sov_oc
                    for vol in hov2_vols:
                        volume += link[vol]
                        person_vol += link[vol]*hov2_oc
                    for vol in hov3_vols:
                        volume += link[vol]
                        person_vol += link[vol]*hov3_oc
                        
                    for vol in truck_vols:
                        truck_volume += link[vol]

                    zones_vkt[i]["all"]["vkt"] += link.length*volume
                    zones_vkt[i]["all"]["truck_vkt"] += link.length*truck_volume
                    zones_vkt[i][shed]["vkt"] += link.length*volume
                    zones_vkt[i][shed]["truck_vkt"] += link.length*truck_volume
                    
                    zones_vkt[i]["all"]["pht"] += person_vol*link.auto_time/60
                    zones_vkt[i]["all"]["trk_pht"]+= truck_volume*link.auto_time/60
                    zones_vkt[i]["all"]["ff_pht"] += person_vol*link.length/link.data2
                    zones_vkt[i]["all"]["ff_trk_pht"]+= truck_volume*link.length/link.data2
                    zones_vkt[i][shed]["pht"] += person_vol*link.auto_time/60
                    zones_vkt[i][shed]["trk_pht"]+= truck_volume*link.auto_time/60
                    zones_vkt[i][shed]["ff_pht"] += person_vol*link.length/link.data2
                    zones_vkt[i][shed]["ff_trk_pht"]+= truck_volume*link.length/link.data2

                    if ( link.auto_volume / (link.data3*link.num_lanes)) > cong:
                        zones_vkt[i]["all"]["cong_vkt"] += link.length*volume
                        zones_vkt[i]["all"]["cong_truck_vkt"] += link.length*truck_volume
                        zones_vkt[i][shed]["cong_vkt"] += link.length*volume
                        zones_vkt[i][shed]["cong_truck_vkt"] += link.length*truck_volume
#     cong_frac = 0.85

#     ntwks_tr = [ntwk_am,ntwk_md]

#     zones_pkt = [{"all":{"pkt": 0.0, "cong_pkt": 0.0},
#                  zones_pd1:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_central:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_east:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_midwest:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_southwest:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_west:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_north:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_niagara:{"pkt": 0.0, "cong_pkt": 0.0}},
#                 {"all":{"pkt": 0.0, "cong_pkt": 0.0},
#                  zones_pd1:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_central:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_east:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_midwest:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_southwest:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_west:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_north:{"pkt": 0.0, "cong_pkt": 0.0},
#                  shed_niagara:{"pkt": 0.0, "cong_pkt": 0.0}}]

#     for i in range(2):
#         for line in ntwks_tr[i].transit_lines():
#             for segment in line.segments():
#                 link = segment.link
#                 add = False 
#                 #This section determines what shed the link belongs to
#                 if ((link.type > 860 and link.type < 900) or (link.type >= 325 and link.type <= 328) or link.type == 330 or link.type == 332): #North
#                     shed = shed_north
#                     add = True
#                 elif ((link.type > 101 and link.type <200) or link.type == 100 or link.type ==333 or link.type ==329 or link.type ==331): #Central
#                     shed = shed_central
#                     add = True
#                 elif link.type == 101: #PD1
#                     shed = zones_pd1
#                     add = True
#                 elif ((link.type >199 and link.type <300) or (link.type > 899 and link.type < 950)): # East
#                     shed = shed_east
#                     add = True
#                 elif link.type == 820 or link.type ==850: # West
#                     shed = shed_west
#                     add = True
#                 elif (link.type >599 and link.type<700) or link.type ==810 or link.type == 800: #Southwest
#                     shed = shed_southwest
#                     add = True
#                 elif link.type == 700: # Niagara
#                     shed = shed_niagara
#                     add = True
#                 elif link.type >= 400 and link.type < 600: # Midwest
#                     shed = shed_midwest
#                     add = True   
#                 if add:
#                     hdw = line.headway
#                     cap = line.vehicle.total_capacity*120/hdw
#                     zones_pkt[i]["all"]["pkt"] += link.length * segment.transit_volume
#                     zones_pkt[i][shed]["pkt"] += link.length * segment.transit_volume
#                     if segment.transit_volume/cap > cong_frac:
#                         zones_pkt[i]["all"]["cong_pkt"] += link.length * segment.transit_volume
#                         zones_pkt[i][shed]["cong_pkt"] += link.length * segment.transit_volume
    
#     freq_th = 8 # high frequency transit threshold of 15 minute service or 4 vehicles per hour
#     dist_th = 850 # 850 m approximate distance can be walked at speed of 5 km/h in 10 minutes
#     nodeDict = []
#     for node in ntwks[1].nodes():
#         freq = 0
#         for segment in node.outgoing_segments():
#             if segment.allow_boardings or segment.allow_alightings:
#                 freq += 60/segment.line.headway
#         if freq > freq_th:
#             nodeDict.append({"ID": node, "x": node.x, "y": node.y})
#     mnodedf = pd.DataFrame.from_dict(nodeDict)
#     centroid_list = []
#     for centroid in ntwks[i].centroids():
#         x = centroid.x
#         y = centroid.y
#         mnodedf["dist"] = ((x - mnodedf["x"]).pow(2) +(y - mnodedf["y"]).pow(2)).pow(0.5)
#         sortdf = mnodedf.sort("dist")
#         if sortdf["dist"].values[0] < dist_th:
#             centroid_list.append({"ID": centroid.id, "Accessible": 1})
#         else:
#             centroid_list.append({"ID": centroid.id, "Accessible": 0})
#     zone_list = pd.DataFrame.from_dict(centroid_list)
#     zone_acc = eb.available_matrix_identifier('ORIGIN')
#     create_matrix(matrix_id = zone_acc,
#         matrix_name= "zone_acc_15mintransit",
#         matrix_description = "zone accessible to 15-min transit within 10 min",
#                  overwrite = True)
#     mat = eb.matrix(zone_acc)
#     mat.set_numpy_data(zone_list.Accessible.values,sc_am)
                        
#     zoneList = []
#     for zone in ntwk_am.centroids():
#         zoneList.append(zone.id)
#     zoneList.sort()
#     shortest_path = eb.available_matrix_identifier('FULL')
#     create_matrix(matrix_id = shortest_path,
#         matrix_name= "shortest.path",
#         matrix_description = "shortest_path")
#     sp = eb.matrix(shortest_path).get_numpy_data(sc_am)
#     #df = pd.DataFrame(data = sp, index = zoneList,columns =zoneList).stack()

#     create_extra("LINK","@ff_time","Free Flow Time", 0,overwrite = True, scenario=sc_am)

#     calc1 = deepcopy(netCalcSpec)
#     calc1["result"] = "@ff_time"
#     calc1["expression"] = "length/ul2*60"
#     calc1["selections"]["link"] = "mode = d"
#     network_calc(calc1,sc_am)

#     calc1["expression"] = "9999"
#     calc1["selections"]["link"] = "not mode = d"
#     network_calc(calc1,sc_am)

#     ntwk_am = sc_am.get_network()

#     for i in range(len(zoneList)):
#         tree = ntwk_am.shortest_path_tree(zoneList[i],"@ff_time")
# #         print("Zone {} started".format(zoneList[i]))
#         for j in range(len(zoneList)):
#             if i != j:
#                 sp[i][j] = tree.cost_to_node(zoneList[j])

#     eb.matrix(shortest_path).set_numpy_data(sp, sc_am)
#     hbw = True
#         hbw_time ={"sov": matrix_list[sov_time_am],
#                 "hov2" : matrix_list[hov2_time_am],
#                 "hov3" : matrix_list[hov3_time_am],
#                 "Express Bus" : transit_time_am["Express Bus"],
#                 "Rapid Bus" : transit_time_am["Rapid Bus"],
#                 "Local Bus" : transit_time_am["Local Bus"],
#                 "Train" : transit_time_am["Train"],
#                 "Subway" : transit_time_am["Subway"],
#                 "walk": matrix_list[active_time],
#                 "bike":matrix_list[active_time] + "*5"}

#         exp_hbw_dem_time = ""
#         exp_hbw_dem = ""
#         for mode in modes.keys():
#             hbw_list[mode] = eb.available_matrix_identifier('FULL')
#             create_matrix(matrix_id = hbw_list[mode],
#                           matrix_name= "hbw_dem_{}".format(mode).replace(" ","_"),
#                           matrix_description = "hbw demand for {}".format(mode))
#             hbw_mat = eb.matrix(hbw_list[mode])
#             first = True
#             sub_modes = modes[mode]
#             for sub_mode in sub_modes:
#                 if first:
#                     tot_mat = df[df["Matrix_Name"] == sub_mode].set_index("_SZX_").iloc[:,3:].values
#                     first = False
#                 else:
#                     tot_mat = tot_mat + df[df["Matrix_Name"] == sub_mode].set_index("_SZX_").iloc[:,3:].values

#             padding = zones - tot_mat.shape[0]
#             padded_mat = np.pad(tot_mat,[(0,padding),(0,padding)],mode='constant')
#             hbw_mat.set_numpy_data(padded_mat,sc_am)
#             exp_hbw_dem_time += "{}*{}+".format(hbw_list[mode],hbw_time[mode])
#             exp_hbw_dem += "{}+".format(hbw_list[mode])

#         exp_hbw_dem_time = exp_hbw_dem_time[:-1]
#         exp_hbw_dem = exp_hbw_dem[:-1]
#     except:
#         print("HBW matrix files not found.. probably")
#         hbw = False
    
    #Actual MOE Script writing happens here
    out_file = r"C:\Users\PECHEN\Desktop\20201016_TEST_MOE_{}_test.csv".format(scen)
    
    moef = open(out_file, 'w')
    moef.write("MOE,MOE_ID," + ",".join(zones_array) + "\n")
    
    moef.write("Peak Auto Trips,010405,")

    for origins in zones_array:
        spec = get_mc_spec(None, exp_pk_pers_trips_auto,"+","+",origins,"all",None)
        top = matrix_calculator(spec)['result']
        moef.write("{0:.4f},".format(top))
    moef.write("\n")
    
    moef.write("Offpeak Auto Trips,010406,")

    for origins in zones_array:
        spec = get_mc_spec(None, exp_offpk_pers_trips_auto,"+","+",origins,"all",None)
        top = matrix_calculator(spec)['result']
        moef.write("{0:.4f},".format(top))
    moef.write("\n")
    
#     moef.write("Offpeak Auto Trips,010406,")

#     for origins in zones_array:
#         spec = get_mc_spec(None, exp_offpk_pers_trips_auto,"+","+",origins,"all",None)
#         top = matrix_calculator(spec)['result']
#         moef.write("{0:.4f},".format(top))
#     moef.write("\n")
    
    moef.write("Daily Transit Trips,010407,")

    for origins in zones_array:
        spec = get_mc_spec(None, exp_daily_pers_transtrips,"+","+",origins,"all",None)
        top = matrix_calculator(spec)['result']
        moef.write("{0:.4f},".format(top))
    moef.write("\n")
    
#     moef.write("Offpeak Transit Trips,010408,")

#     for origins in zones_array:
#         spec = get_mc_spec(None, exp_offpk_transpers_trips,"+","+",origins,"all",None)
#         top = matrix_calculator(spec)['result']
#         moef.write("{0:.4f},".format(top))
#     moef.write("\n")
    
    moef.write("Daily Active Trips,010409,")

    for origins in zones_array:
        spec = get_mc_spec(None, exp_daily_pers_activetrips,"+","+",origins,"all",None)
        top = matrix_calculator(spec)['result']
        moef.write("{0:.4f},".format(top))
    moef.write("\n")
    
#     moef.write("Offpeak Active Trips,010410,")

#     for origins in zones_array:
#         spec = get_mc_spec(None, exp_offpk_active_pers_trips,"+","+",origins,"all",None)
#         top = matrix_calculator(spec)['result']
#         moef.write("{0:.4f},".format(top))
#     moef.write("\n")
    
    moef.write("AM Peak Auto Speed,010302,")

    for origins in zones_array:
        spec = get_mc_spec(None, exp_pk_pers_autotrip_speed,"+","+",origins,"all",None)
        top = matrix_calculator(spec)['result']
        moef.write("{0:.4f},".format(top))
    moef.write("\n")
    
#     moef.write("AM Peak Transit mode share,010302,")

#     MOE_fraction(moef,matrix_list[transit_demand_pk],exp_pk_pers_trips,zones_array)

#     moef.write("Person Hours lost to congestion ,080405,")

#     for origins in zones_array:
#         pht_lost = 0
#         for i in range(2):
#             pht_lost +=  zones_vkt[0][origins]["pht"] - zones_vkt[0][origins]["ff_pht"]
#         moef.write("{0:.4f},".format(pht_lost))
#     moef.write("\n")
    
    moef.close()
    
    print("Finished Processing {}".format(scen))

Now processing 2016_prefnet_OGS
Finished Processing 2016_prefnet_OGS
